In [2]:
import pandas as pd
xls = pd.ExcelFile("23-RC-Pub-Data-Set.xlsx")
xls.sheet_names


['Revision History',
 'General',
 'Finance',
 'ELA Math Science',
 'IAR',
 'IAR (2)',
 'SAT',
 'ISA',
 'DLM-AA',
 'DLM-AA (2)',
 'CTE',
 'TeacherOutofField',
 'Discipline']

Loops through every column name.

Keeps columns if they contain the words “grad” or “dropout”.

Helps you automatically find relevant variables → no hand-picking needed.

Calls the 2023 ACS API (subject table S1903) for Illinois counties (state=17).

Retrieves:

County name

Median household income

FIPS codes

Renames columns into readable labels.

Saves the income dataset as CSV.

In [ ]:
url = (
    "https://api.census.gov/data/2023/acs/acs5/subject?"
    "get=NAME,S1901_C01_012E&for=county:*&in=state:17"
)

#load income data
income_df = pd.read_json(url)
income_df.columns = ["County_Name", "Median_Income", "State_FIPS", "County_FIPS"]

#remove the first row which contains header information
income_df = income_df.iloc[1:].copy()

#clean county names to match education dataset
income_df["County_Clean"] = (
    income_df["County_Name"]
    .str.replace(" County, Illinois", "", regex=False)
    .str.upper()
    .str.strip()
)

#convert median income to integer
income_df["Median_Income"] = income_df["Median_Income"].astype(int)

income_df.head()


,County_Name,Median_Income,State_FIPS,County_FIPS,County_Clean
1,"Adams County, Illinois",64962,17,001,ADAMS
2,"Alexander County, Illinois",43523,17,003,ALEXANDER
3,"Bond County, Illinois",61603,17,005,BOND
4,"Boone County, Illinois",81638,17,007,BOONE
5,"Brown County, Illinois",72288,17,009,BROWN


Loads the entire school dataset (default sheet).

Loads income dataset again.

Renames columns to something nicer.

Removes the text " County, Illinois"
Example: "Cook County, Illinois" → "Cook"

Strips extra spaces.

Converts to uppercase → standardizes for merging.

Reads and prints the first few rows of each important sheet.

Shows available columns → helps you decide what to extract.

Loads each sheet into a separate DataFrame.

Creates a helper function that returns all columns containing any of the keywords.

In [ ]:
xls = pd.ExcelFile("23-RC-Pub-Data-Set.xlsx")

#split each sheet into separate dataframes
general = pd.read_excel(xls, sheet_name="General")
finance = pd.read_excel(xls, sheet_name="Finance")
ela_math = pd.read_excel(xls, sheet_name="ELA Math Science")

#create a function that finds columns based on keywords
def find_columns(df, keywords):
    found = []
    for col in df.columns:
        for word in keywords:
            if word.lower() in col.lower():
                found.append(col)
                break
    return found

#use find_columns function to find relevant columns
county_cols = find_columns(general, ["county"])
grad_cols = find_columns(general, ["graduation"])
dropout_cols = find_columns(general, ["dropout"])
income_related_cols = find_columns(finance, ["expenditure", "revenue"])
achievement_cols = find_columns(ela_math, ["proficiency", "growth"])

#print found columns
print("County columns:\n", county_cols, "\n")
print("Graduation columns:\n", grad_cols, "\n")
print("Dropout columns:\n", dropout_cols, "\n")
print("Finance columns:\n", income_related_cols, "\n")
print("Achievement columns:\n", achievement_cols, "\n")



County columns:
 ['County'] 

Graduation columns:
 ['High School 4-Year Graduation Rate - Total', 'High School 4-Year Graduation Rate - Male', 'High School 4-Year Graduation Rate - Female', 'High School 4-Year Graduation Rate - White', 'High School 4-Year Graduation Rate - Black or African American', 'High School 4-Year Graduation Rate - Hispanic or Latino', 'High School 4-Year Graduation Rate - Asian', 'High School 4-Year Graduation Rate - Native Hawaiian or Other Pacific Islander', 'High School 4-Year Graduation Rate - American Indian or Alaska Native', 'High School 4-Year Graduation Rate - Two or More Races', 'High School 4-Year Graduation Rate - Children with Disabilities', 'High School 4-Year Graduation Rate - IEP', 'High School 4-Year Graduation Rate - EL', 'High School 4-Year Graduation Rate - Low Income', 'High School 4-Year Graduation Rate - Homeless', 'High School 4-Year Graduation Rate - Migrant', 'High School 4-Year Graduation Rate - Youth In Care', 'High School 4-Year Grad

Keeps only the columns needed for the analysis.

Drops everything else (demographics, school-level details, etc.).

In [ ]:
#select relevant columns
general_subset = general[["County", 
                          "High School 4-Year Graduation Rate - Total",
                          "High School Dropout Rate - Total"]]

finance_subset = finance[["County",
                          "$ Total Per-Pupil Expenditures - Federal",
                          "$ Total Per-Pupil Expenditures - State/Local"]]

ela_math_subset = ela_math[["County",
                            "% ELA Proficiency",
                            "% Math Proficiency",
                            "% Science Proficiency"]]


Some counties have multiple districts.

groupby("County").mean() aggregates them.

Produces one row per county for each measure.

In [6]:
# Group by County and compute averages
general_county = general_subset.groupby("County").mean(numeric_only=True).reset_index()
finance_county = finance_subset.groupby("County").mean(numeric_only=True).reset_index()
ela_math_county = ela_math_subset.groupby("County").mean(numeric_only=True).reset_index()


### Merge the three education datasets

In [7]:
#merged general_county, finance_county, and ela_math_county dataframes
merged_edu = general_county.merge(finance_county, on="County", how="outer")
merged_edu = merged_edu.merge(ela_math_county, on="County", how="outer")

print(merged_edu.head())


      County  High School 4-Year Graduation Rate - Total  \
0      Adams                                       88.00   
1  Alexander                                       80.55   
2       Bond                                       82.80   
3      Boone                                       86.42   
4      Brown                                       98.00   

   High School Dropout Rate - Total  $ Total Per-Pupil Expenditures - Federal  \
0                              6.10                               1757.251667   
1                               NaN                               8071.032857   
2                              2.90                                839.792000   
3                              2.56                               1079.741667   
4                               NaN                                664.745000   

   $ Total Per-Pupil Expenditures - State/Local  % ELA Proficiency  \
0                                  11029.023333          31.481818   
1           

Standardizes both datasets’ county names (so “Cook” and “COOK” match).

Joins school performance and Census income data by county.

Keeps only relevant columns (e.g., graduation rate, dropout rate, expenditures, proficiency, and median income).

In [8]:
# Clean both county columns to match (uppercase and strip spaces)
merged_edu["County_Clean"] = merged_edu["County"].str.upper().str.strip()

# Merge income data with education data
final = pd.merge(
    merged_edu,
    income_df[["County_Clean", "Median_Income"]],
    on="County_Clean",
    how="left"
)

# Drop redundant columns if needed
final = final.drop(columns=["County"])
final


,High School 4-Year Graduation Rate - Total,High School Dropout Rate - Total,$ Total Per-Pupil Expenditures - Federal,$ Total Per-Pupil Expenditures - State/Local,% ELA Proficiency,% Math Proficiency,% Science Proficiency,County_Clean,Median_Income
0,88.000000,6.100000,1757.251667,11029.023333,31.481818,27.440909,54.680000,ADAMS,64962.0
1,80.550000,NaN,8071.032857,14312.575714,5.928571,2.414286,18.414286,ALEXANDER,43523.0
2,82.800000,2.900000,839.792000,12631.335000,32.466667,22.900000,52.355556,BOND,61603.0
3,86.420000,2.560000,1079.741667,14493.197222,24.572222,17.922222,45.726667,BOONE,81638.0
4,98.000000,NaN,664.745000,11466.562500,27.575000,30.025000,66.133333,BROWN,72288.0
...,...,...,...,...,...,...,...,...,...
99,87.633333,3.050000,1760.558919,13361.835405,36.796774,26.609677,58.800000,WHITESIDE,64536.0
100,91.537037,2.278947,1002.377796,15191.243978,38.582759,30.131034,54.185526,WILL,107799.0
101,85.280000,4.360000,2049.546154,10263.043462,38.762500,30.254167,56.495455,WILLIAMSON,65521.0
102,80.482353,4.753846,1667.015978,13640.608043,29.525610,22.519512,43.760526,WINNEBAGO,64363.0


In [9]:
#Rename columns for clarity
final = final.rename(columns={
    "$ Total Per-Pupil Expenditures - Federal":     "PerPupil_Federal",
    "$ Total Per-Pupil Expenditures - State/Local": "PerPupil_StateLocal",
    "% ELA Proficiency":                            "ELA_Proficiency",
    "% Math Proficiency":                           "Math_Proficiency",
    "% Science Proficiency":                        "Science_Proficiency",
    "High School 4-Year Graduation Rate - Total":   "GradRate_Total",
    "High School Dropout Rate - Total":             "DropoutRate_Total"
})
final.head()


,GradRate_Total,DropoutRate_Total,PerPupil_Federal,PerPupil_StateLocal,ELA_Proficiency,Math_Proficiency,Science_Proficiency,County_Clean,Median_Income
0,88.00,6.10,1757.251667,11029.023333,31.481818,27.440909,54.680000,ADAMS,64962.0
1,80.55,NaN,8071.032857,14312.575714,5.928571,2.414286,18.414286,ALEXANDER,43523.0
2,82.80,2.90,839.792000,12631.335000,32.466667,22.900000,52.355556,BOND,61603.0
3,86.42,2.56,1079.741667,14493.197222,24.572222,17.922222,45.726667,BOONE,81638.0
4,98.00,NaN,664.745000,11466.562500,27.575000,30.025000,66.133333,BROWN,72288.0


In [10]:
percent_cols = [
    "GradRate_Total",
    "DropoutRate_Total",
    "ELA_Proficiency",
    "Math_Proficiency",
    "Science_Proficiency"
]

for col in percent_cols:
    final[col] = final[col] / 100.0


In [11]:
final.head()

,GradRate_Total,DropoutRate_Total,PerPupil_Federal,PerPupil_StateLocal,ELA_Proficiency,Math_Proficiency,Science_Proficiency,County_Clean,Median_Income
0,0.8800,0.0610,1757.251667,11029.023333,0.314818,0.274409,0.546800,ADAMS,64962.0
1,0.8055,NaN,8071.032857,14312.575714,0.059286,0.024143,0.184143,ALEXANDER,43523.0
2,0.8280,0.0290,839.792000,12631.335000,0.324667,0.229000,0.523556,BOND,61603.0
3,0.8642,0.0256,1079.741667,14493.197222,0.245722,0.179222,0.457267,BOONE,81638.0
4,0.9800,NaN,664.745000,11466.562500,0.275750,0.300250,0.661333,BROWN,72288.0


In [12]:
final[["GradRate_Total",
    "DropoutRate_Total",
    "PerPupil_Federal",
    "PerPupil_StateLocal",
    "ELA_Proficiency",
    "Math_Proficiency",
    "Science_Proficiency",
    "Median_Income"]].isna().sum()


GradRate_Total          0
DropoutRate_Total      16
PerPupil_Federal        1
PerPupil_StateLocal     1
ELA_Proficiency         0
Math_Proficiency        0
Science_Proficiency     1
Median_Income           5
dtype: int64

In [13]:
#Impute columns with only 1 missing with median 
# columns safe to impute with median (tiny number of missing)
median_impute_cols = [
    "PerPupil_Federal",
    "PerPupil_StateLocal",
    "PerPupil_Total",
    "Science_Proficiency"
]

for col in median_impute_cols:
    if col in final.columns:
        final[col] = final[col].fillna(final[col].median())


In [14]:
#Dropping missing values fromo DropoutRate_Total and Median_Income
final_clean = final.dropna(subset=["DropoutRate_Total", "Median_Income"]).copy()
final_clean[[
    "GradRate_Total",
    "DropoutRate_Total",
    "PerPupil_Federal",
    "PerPupil_StateLocal",
    "ELA_Proficiency",
    "Math_Proficiency",
    "Science_Proficiency",
    "Median_Income"
]].isna().sum()



GradRate_Total         0
DropoutRate_Total      0
PerPupil_Federal       0
PerPupil_StateLocal    0
ELA_Proficiency        0
Math_Proficiency       0
Science_Proficiency    0
Median_Income          0
dtype: int64

In [ ]:
#move County_Clean to the first column
cols = ["County_Clean"] + [c for c in final.columns if c != "County_Clean"]
final = final[cols]


In [16]:
final_clean.to_csv("illinois_cleaned_county_data.csv", index=False)
